<a href="https://colab.research.google.com/github/koyal2004/Innomatics-Research-Lab-Hackathon/blob/main/Innomatics_Labs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import json
import sqlite3


In [2]:

orders = pd.read_csv("/content/orders.csv")

with open("users.json", "r") as f:
    users = pd.DataFrame(json.load(f))

conn = sqlite3.connect(":memory:")
with open("restaurants.sql", "r") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)


In [3]:
df = orders.merge(users, on="user_id", how="left")

df = df.merge(restaurants, on="restaurant_id", how="left")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [4]:
df.to_csv("combined_data.csv", index=False)


In [5]:
df[df["membership"] == "Gold"] \
  .groupby("city")["total_amount"] \
  .sum() \
  .sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [6]:
df.groupby("cuisine")["total_amount"] \
  .mean() \
  .sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [7]:
(df.groupby("user_id")["total_amount"].sum() > 1000).sum()


np.int64(2544)

In [8]:
bins = [0, 3.5, 4.0, 4.5, 5.0]
labels = ["<3.5", "3.5–4.0", "4.0–4.5", "4.5–5.0"]

df["rating_range"] = pd.cut(df["rating"], bins=bins, labels=labels)

df.groupby("rating_range")["total_amount"] \
  .sum() \
  .sort_values(ascending=False)

/tmp/ipython-input-2284403027.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"] \


,total_amount
rating_range,
4.5–5.0,2197030.75
<3.5,2136772.70
4.0–4.5,1960326.26
3.5–4.0,1717494.41


In [9]:
df[df["membership"] == "Gold"] \
  .groupby("city")["total_amount"] \
  .mean() \
  .sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [10]:
df.groupby("cuisine").agg(
    distinct_restaurants=("restaurant_id", "nunique"),
    total_revenue=("total_amount", "sum")
).sort_values("distinct_restaurants")


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [11]:
round(
    (df[df["membership"] == "Gold"].shape[0] / df.shape[0]) * 100
)


50

In [12]:
df.groupby("restaurant_name_x").agg(
    order_count=("order_id", "count"),
    avg_order_value=("total_amount", "mean")
).query("order_count < 20") \
 .sort_values("avg_order_value", ascending=False) \
 .head(1)


,order_count,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308


In [13]:
df.groupby(["membership","cuisine"])["total_amount"] \
  .sum() \
  .sort_values(ascending=False)



membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [ ]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)
df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"] \
  .sum() \
  .sort_values(ascending=False)

,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [14]:
df[df["membership"] == "Gold"].shape[0]


4987

In [15]:
round(df[df["city"] == "Hyderabad"]["total_amount"].sum())


1889367

In [16]:
df["user_id"].nunique()


2883

In [17]:
round(
    df[df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

In [18]:
orders_high_rating = df[df["rating"] >= 4.5].shape[0]

gold = df[df["membership"] == "Gold"]
top_city = gold.groupby("city")["total_amount"].sum().idxmax()

orders_top_city_gold = gold[gold["city"] == top_city].shape[0]

orders_high_rating, top_city, orders_top_city_gold



(3374, 'Chennai', 1337)

In [19]:
set(orders.columns).intersection(set(users.columns))


{'user_id'}

In [20]:
df.shape[0]


10000

In [21]:
df[df["name"].isna()].head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,rating_range


In [22]:
users.columns


Index(['user_id', 'name', 'city', 'membership'], dtype='object')

In [23]:
restaurants["cuisine"].unique()


array(['Chinese', 'Indian', 'Mexican', 'Italian'], dtype=object)

In [24]:
df["user_id"].value_counts().head()


,count
user_id,
2973,13
1515,12
874,11
496,11
1337,11
